In [2]:
#!/usr/bin/env python
import torch
import torchvision.datasets
import torchvision.transforms as transforms
from torch.utils.data import dataset
from torch import nn
# from torch.nn.modules import upsampling
# from torch.functional import F
from torch.optim import Adam

image_transform = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize((.5, .5, .5),
                         (.5, .5, .5))
])


def get_data_loader(dataset_location, batch_size):
    trainvalid = torchvision.datasets.SVHN(
        dataset_location, split='train',
        download=True,
        transform=image_transform
    )

    trainset_size = int(len(trainvalid) * 0.9)
    trainset, validset = dataset.random_split(
        trainvalid,
        [trainset_size, len(trainvalid) - trainset_size]
    )

    trainloader = torch.utils.data.DataLoader(
        trainset,
        batch_size=batch_size,
        shuffle=True,
        num_workers=2
    )

    validloader = torch.utils.data.DataLoader(
        validset,
        batch_size=batch_size,
    )

    testloader = torch.utils.data.DataLoader(
        torchvision.datasets.SVHN(
            dataset_location, split='test',
            download=True,
            transform=image_transform
        ),
        batch_size=batch_size,
    )

    return trainloader, validloader, testloader


class Classifier(nn.Module):
    def __init__(self):
        super(Classifier, self).__init__()
        self.conv_stack = nn.Sequential(
            nn.Conv2d(3, 8, 3, padding=1),
            nn.ELU(),
            nn.Dropout2d(p=0.1),
            nn.Conv2d(8, 16, 3, padding=1),
            nn.ELU(),
            nn.Dropout2d(p=0.1),
            nn.MaxPool2d(2),

            nn.Conv2d(16, 16, 3, padding=1),
            nn.ELU(),
            nn.Dropout2d(p=0.1),
            nn.Conv2d(16, 32, 3, padding=1),
            nn.ELU(),
            nn.Dropout2d(p=0.1),
            nn.MaxPool2d(2),

            nn.Conv2d(32, 64, 3, padding=1),
            nn.ELU(),
            nn.Dropout2d(p=0.1),
            nn.Conv2d(64, 128, 3, padding=1),
            nn.ELU(),
            nn.Dropout2d(p=0.1),
            nn.MaxPool2d(2),

            nn.Conv2d(128, 512, 2),
        )

        self.mlp = nn.Sequential(
            nn.ELU(),
            nn.Dropout(0.5),
            nn.Linear(512, 10),
        )
        # for p in self.parameters():
        #     if p.dim() > 1:
        #         nn.init.xavier_uniform_(p)

    def forward(self, x):
        return self.mlp(self.extract_features(x))

    def extract_features(self, x):
        return self.conv_stack(x)[:, :, 0, 0]


def evaluate(classify, dataset):
    with torch.no_grad():
        classify.eval()
        correct = 0.
        total = 0.
        for x, y in dataset:
            if cuda:
                x = x.cuda()
                y = y.cuda()

            c = (classify(x).argmax(dim=-1) == y).sum().item()
            t = x.size(0)
            correct += c
            total += t
    acc = correct / float(total)
    return acc


if __name__ == "__main__":
    train, valid, test = get_data_loader("svhn", 32)
    classify = Classifier()
    params = classify.parameters()
    optimizer = Adam(params)
    ce = nn.CrossEntropyLoss()
    best_acc = 0.
    cuda = torch.cuda.is_available()
    if cuda:
        classify = classify.cuda()

    for _ in range(50):
        classify.train()
        for i, (x, y) in enumerate(train):
            if cuda:
                x = x.cuda()
                y = y.cuda()
            out = classify(x)
            loss = ce(out, y)
            loss.backward()
            optimizer.step()
            optimizer.zero_grad()
            if (i + 1) % 200 == 0:
                print(loss.item())
        acc = evaluate(classify, valid)
        print("Validation acc:", acc,)

        if acc > best_acc:
            best_acc = acc
            torch.save(classify, "svhn_classifier.pt")
            print("Saved.")
    classify = torch.load("svhn_classifier.pt")
    print("Test accuracy:", evaluate(classify, test))


Using downloaded and verified file: svhn/train_32x32.mat
Using downloaded and verified file: svhn/test_32x32.mat
2.0147299766540527
0.7511980533599854
0.6944493055343628
0.44535911083221436
0.5588081479072571
0.522225558757782
0.7604825496673584
0.33995765447616577
0.46941715478897095
0.7004664540290833
Validation acc: 0.8423423423423423
Saved.
0.730006217956543
0.3871992230415344
0.40804022550582886
0.24335086345672607
0.40102720260620117
0.947547972202301
0.31539034843444824
1.138545274734497
0.3815324604511261
0.1509629786014557
Validation acc: 0.8838383838383839
Saved.
0.324197381734848
0.3735715448856354
0.3417072296142578
0.4338708519935608
0.45502370595932007
0.3421054482460022
0.4026845395565033
0.3229350745677948
0.4506441652774811
0.22691258788108826
Validation acc: 0.8901173901173901
Saved.
1.1068758964538574
0.2601611316204071
0.20246635377407074
0.500021755695343
0.6179907321929932
0.9445339441299438
0.35724925994873047
0.19908244907855988
0.15368063747882843
0.19225752353

Validation acc: 0.9194649194649195
0.16691076755523682
0.37964147329330444
0.2035020887851715
0.357166051864624
0.3247738778591156
0.636227548122406
0.8125154972076416
0.559807538986206
0.6432245969772339
0.28976839780807495
Validation acc: 0.9190554190554191
0.06372131407260895
0.30168917775154114
0.5170081853866577
0.25809070467948914
0.1578112542629242
0.14530253410339355
0.2480258345603943
0.06452395021915436
0.3844338059425354
0.3299834430217743
Validation acc: 0.9183729183729183
0.21906732022762299
0.8524824380874634
0.2351129800081253
0.41838520765304565
0.45899611711502075
0.22523020207881927
0.4946518540382385
0.29778486490249634
1.1761549711227417
0.019085675477981567
Validation acc: 0.916052416052416
0.42875704169273376
0.0351792573928833
0.5592100620269775
0.3200380504131317
0.2433391958475113
0.21406137943267822
0.2084033340215683
0.24686311185359955
0.18424874544143677
0.07273814082145691
Validation acc: 0.9125034125034125
0.23843270540237427
0.26399359107017517
0.1718524